In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
df_access = spark.read.csv(path="access_logs_csv.csv", sep=";", header=True)
df_browser_site = spark.read.csv(path="official_browser_site_csv.csv", sep=";", header=True)
df_dict_site = spark.read.csv(path="dict_site_status_code_csv.csv", sep=";", header=True)

df_access.show()
df_browser_site.show()
df_dict_site.show()

+--------------+--------------------+----------------+--------------------+--------------------+
|     ip_adress|           date_time|site_status_code|                site|         browser_raw|
+--------------+--------------------+----------------+--------------------+--------------------+
| 209.160.24.63|03/Mar/2016 18:22:16|             200|http://www.google...|Mozilla/5.0 (Wind...|
| 209.160.24.63|03/Mar/2016 18:22:16|             200|http://www.explor...|Mozilla/5.0 (Wind...|
| 209.160.24.63|03/Mar/2016 18:22:17|             200|http://www.explor...|Mozilla/5.0 (Wind...|
| 209.160.24.63|03/Mar/2016 18:22:19|             200|http://www.explor...|Mozilla/5.0 (Wind...|
| 209.160.24.63|03/Mar/2016 18:22:20|             200|http://www.explor...|Mozilla/5.0 (Wind...|
| 209.160.24.63|03/Mar/2016 18:22:20|             200|http://www.explor...|Mozilla/5.0 (Wind...|
| 209.160.24.63|03/Mar/2016 18:22:21|             200|http://www.explor...|Mozilla/5.0 (Wind...|
| 209.160.24.63|03/Mar/2016 18

In [32]:
df1 = df_access.join(df_dict_site, df_access.site_status_code == df_dict_site.site_status_code, "inner").select(
    df_access.ip_adress, df_access.date_time, df_access.site, df_access.browser_raw, df_dict_site["*"]
)
df1 = df1.join(df_browser_site, df_browser_site.browser.substr(1, 5) == df1.browser_raw.substr(1, 5), "inner").select(
    df1.ip_adress, df1.date_time, df1.site, df1.site_status_code, df1.meaning, df_browser_site["*"]
)

df1.show(n=10, truncate=False)

+-------------+--------------------+-------------------------------------------------------------------------------------------------------------+----------------+-------+-------+---------------------+
|ip_adress    |date_time           |site                                                                                                         |site_status_code|meaning|browser|official_browser_site|
+-------------+--------------------+-------------------------------------------------------------------------------------------------------------+----------------+-------+-------+---------------------+
|209.160.24.63|03/Mar/2016 18:22:16|http://www.google.com                                                                                        |200             |OK     |Mozilla|www.mozilla.org      |
|209.160.24.63|03/Mar/2016 18:22:16|http://www.exploratorystore.io/oldlink?itemId=EST-6                                                          |200             |OK     |Mozilla|www.mozilla.o

In [33]:
print("Rows cound:", df1.count())
df2 = df1.dropna()
print("Rows with na dropped:", df1.count() - df2.count())
print("Rows in result:", df2.count())

Rows cound: 13628
Rows with na dropped: 242
Rows in result: 13386


In [34]:
addresses = df2.groupBy("site", "ip_adress").count()
filtered_addresses = addresses.filter(addresses["count"] >= 5).select(addresses.site, addresses["count"]).distinct()
filtered_addresses.show(truncate=False)

+------------------------------------------------------------------+-----+
|site                                                              |count|
+------------------------------------------------------------------+-----+
|http://www.google.com                                             |19   |
|http://www.exploratorystore.io                                    |5    |
|http://www.exploratorystore.io/product.screen?productId=DC-SG-G02 |5    |
|http://www.exploratorystore.io/category.screen?categoryId=NULL    |7    |
|http://www.exploratorystore.io/product.screen?productId=WC-SH-G04 |7    |
|http://www.exploratorystore.io/category.screen?categoryId=STRATEGY|6    |
|http://www.exploratorystore.io/category.screen?categoryId=STRATEGY|8    |
|http://www.exploratorystore.io/category.screen?categoryId=ARCADE  |10   |
|http://www.exploratorystore.io                                    |6    |
|http://www.exploratorystore.io/product.screen?productId=SF-BVS-G01|6    |
|http://www.exploratoryst

In [35]:
sorted_addresses = filtered_addresses.orderBy(filtered_addresses["count"].desc()).take(10)
sorted_addresses

[Row(site='http://www.exploratorystore.io', count=20),
 Row(site='http://www.google.com', count=19),
 Row(site='http://www.exploratorystore.io/category.screen?categoryId=ACCESSORIES', count=18),
 Row(site='http://www.exploratorystore.io/category.screen?categoryId=STRATEGY', count=17),
 Row(site='http://www.exploratorystore.io/category.screen?categoryId=STRATEGY', count=15),
 Row(site='http://www.google.com', count=12),
 Row(site='http://www.exploratorystore.io/category.screen?categoryId=STRATEGY', count=12),
 Row(site='http://www.exploratorystore.io/category.screen?categoryId=ARCADE', count=12),
 Row(site='http://www.exploratorystore.io/category.screen?categoryId=STRATEGY', count=11),
 Row(site='http://www.exploratorystore.io/product.screen?productId=SF-BVS-G01', count=11)]

In [36]:
most_popular_between_time = df2.filter(df2.date_time.substr(1, 2) < "11").filter("06" <= df2.date_time.substr(1, 2)).select(df2.site, df2.date_time)
most_popular_between_time = most_popular_between_time.groupBy(most_popular_between_time.site).count()
most_popular_between_time = most_popular_between_time.orderBy(most_popular_between_time["count"].desc()).take(10)
most_popular_between_time

[Row(site='http://www.exploratorystore.io/category.screen?categoryId=STRATEGY', count=559),
 Row(site='http://www.exploratorystore.io', count=422),
 Row(site='http://www.google.com', count=377),
 Row(site='http://www.exploratorystore.io/category.screen?categoryId=ARCADE', count=301),
 Row(site='http://www.exploratorystore.io/category.screen?categoryId=NULL', count=259),
 Row(site='http://www.exploratorystore.io/category.screen?categoryId=ACCESSORIES', count=258),
 Row(site='http://www.exploratorystore.io/product.screen?productId=SF-BVS-G01', count=244),
 Row(site='http://www.exploratorystore.io/category.screen?categoryId=TEE', count=228),
 Row(site='http://www.yahoo.com', count=186),
 Row(site='http://www.exploratorystore.io/category.screen?categoryId=SHOOTER', count=178)]